In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from skopt.space import Integer
from numpy import mean
from skopt.utils import use_named_args

#
from sklearn.linear_model import ElasticNet

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, median_absolute_error

In [ ]:
#dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')
dataset = pd.read_csv('./Dataset/para.csv')
# 
features = ["Freq"]
targets = ["W", "L", "Lf"]
x = dataset[features]
y = dataset[targets]

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [4]:
search_space = [Integer(1, 10, name='max_iter'), Integer(2, 10, name='alpha')]

In [ ]:
@use_named_args(search_space)
def evaluate_model(**params):
    model = ElasticNet(max_iter=params['max_iter'], alpha=params['alpha'])
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    error = mean((y_test - y_pred) ** 2)  # MSE
    return error

In [6]:
result = gp_minimize(evaluate_model, search_space)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 2] before, using random point [4, 3]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 2] before, using random point [1, 7]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 2] before, using random point [7, 8]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\o

In [7]:
best_max_iter = result.x[0]
best_alpha = result.x[1]
best_model = ElasticNet(max_iter=best_max_iter, alpha=best_alpha)
best_model.fit(x_train_scaled, y_train)


ElasticNet(alpha=2, max_iter=9)

In [ ]:
new_dataset = pd.read_csv('./Dataset/para.csv')  #
new_data = new_dataset["Freq"].values.reshape(-1, 1)  # 

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(x_test_scaled)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: max_iter=%d, alpha=%.2f' % (result.x[0], result.x[1]))

# 

print("Predictions:", y_pred)

Best Accuracy: -224.434
Best Parameters: max_iter=9, alpha=2.00
Predictions: [[34.93998214 27.00207998 15.85848074]
 [46.53672516 36.01336677 20.72717086]
 [53.47044214 41.40123404 23.6381709 ]
 [43.00904459 33.27217114 19.24613575]
 [34.84537002 26.92856133 15.81875949]
 [23.14725686 17.83850455 10.90751089]
 [36.10235965 27.9053092  16.34648464]
 [45.30676757 35.05762423 20.21079463]
 [38.11624626 29.47020632 17.19197978]
 [44.82019094 34.67952828 20.00651393]
 [43.15772079 33.38770046 19.30855485]
 [52.92980144 40.98112743 23.41119233]
 [41.71150691 32.26391527 18.70138721]
 [43.34694503 33.53473777 19.38799735]
 [34.11550507 26.3614174  15.51233843]
 [52.57838499 40.70805814 23.26365627]
 [36.31861593 28.07335184 16.43727606]
 [42.75224026 33.0726205  19.13832093]
 [37.06199689 28.65099843 16.74937159]
 [34.89943409 26.97057199 15.84145734]
 [46.68540136 36.12889609 20.78958996]
 [41.61689479 32.19039662 18.66166596]
 [23.51894735 18.12732785 11.06355865]
 [23.04588673 17.75973456 

In [ ]:
FreqInput = float(input('Nhập vào tần số muốn dự đoán: '))

# 
new_data = [[FreqInput]]

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(new_data_scaled)

# 
print('Giá trị nhập vào là: ', FreqInput)
print('Giá trị dự đoán:', y_pred)

In [ ]:
def regression_report(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)
    median_ae = median_absolute_error(y_true, y_pred)

    # Mean Bias Deviation (MBD)
    mbd = np.mean(y_pred - y_true)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Symmetric Mean Absolute Percentage Error (sMAPE)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

    report = pd.DataFrame({
        'Metric': [
            'Mean Absolute Error (MAE)',
            'Mean Squared Error (MSE)',
            'Root Mean Squared Error (RMSE)',
            'R² Score',
            'Explained Variance Score',
            'Median Absolute Error',
            'Mean Bias Deviation (MBD)',
            'Mean Absolute Percentage Error (MAPE)',
            'Symmetric Mean Absolute Percentage Error (sMAPE)'
        ],
        'Gausians': [mae, mse, rmse, r2, explained_variance, median_ae, mbd, mape, smape]
    })

    return report

report = regression_report(y_test, y_pred)
print(report)

                                             Metric    Gausians
0                         Mean Absolute Error (MAE)    8.903173
1                          Mean Squared Error (MSE)  225.433612
2                    Root Mean Squared Error (RMSE)   15.014447
3                                          R² Score    0.446929
4                          Explained Variance Score    0.452539
5                             Median Absolute Error    4.881884
6                         Mean Bias Deviation (MBD)   -1.450768
7             Mean Absolute Percentage Error (MAPE)   27.124767
8  Symmetric Mean Absolute Percentage Error (sMAPE)   26.454043
